In [18]:
import os
import pickle
import numpy as np

In [2]:
basedir = '/home/gamaga/work/cse517-lama/output/results'

In [24]:
model_names = sorted(os.listdir(basedir))
print(model_names)

['bert_base', 'bert_large', 'elmo', 'elmo5B', 'transformerxl']


In [25]:
google_re_templates = ['date_of_birth', 'place_of_birth', 'place_of_death']

In [27]:
# Google RE:
for model_name in model_names:
    print(model_name)
    precisions = []
    for template in google_re_templates:
        filename = os.path.join(basedir, model_name, template, 'result.pkl')
        results = pickle.load(open(filename, 'rb'))
        num_queries = len(results['list_of_results'])
        sum_precision = sum([result['sample_Precision1'] for result in results['list_of_results']])
        avg_precision = sum_precision/num_queries
        print('  Template: %s. Num queries: %d. Avg P@1: %f' % (template, num_queries, avg_precision))
        precisions.append(avg_precision)

    print('  ' + '='*30)
    print('  Total P@1: %f' % np.mean(precisions))

bert_base
  Template: date_of_birth. Num queries: 1825. Avg P@1: 0.015342
  Template: place_of_birth. Num queries: 2937. Avg P@1: 0.149472
  Template: place_of_death. Num queries: 765. Avg P@1: 0.130719
  Total P@1: 0.098511
bert_large
  Template: date_of_birth. Num queries: 1825. Avg P@1: 0.013699
  Template: place_of_birth. Num queries: 2937. Avg P@1: 0.160708
  Template: place_of_death. Num queries: 765. Avg P@1: 0.139869
  Total P@1: 0.104759
elmo
  Template: date_of_birth. Num queries: 1825. Avg P@1: 0.000000
  Template: place_of_birth. Num queries: 2937. Avg P@1: 0.054818
  Template: place_of_death. Num queries: 765. Avg P@1: 0.000000
  Total P@1: 0.018273
elmo5B
  Template: date_of_birth. Num queries: 1825. Avg P@1: 0.000000
  Template: place_of_birth. Num queries: 2937. Avg P@1: 0.075247
  Template: place_of_death. Num queries: 765. Avg P@1: 0.000000
  Total P@1: 0.025082
transformerxl
  Template: date_of_birth. Num queries: 1825. Avg P@1: 0.008219
  Template: place_of_birth. N

In [28]:
# ConceptNet and Squad
template='test'
filename = os.path.join(basedir, model_name, template, 'result.pkl')
results = pickle.load(open(filename, 'rb'))

In [32]:
results['list_of_results'][0].keys()

dict_keys(['sample', 'uuid', 'token_ids', 'masked_indices', 'label_index', 'masked_topk', 'sample_MRR', 'sample_Precision', 'sample_perplexity', 'sample_Precision1'])